# <center>DATA.ML.200 Pattern Recognition and Machine Learning</center>

## <center>Assignment 1</center>
## <center>Team Members: 1. Chouaib Benchraka  2. Md. Abdullah-Al Mamun</center>

## LOADING DATA

In [2]:
!unzip tau-ethiopic-digit-recognition.zip

Archive:  tau-ethiopic-digit-recognition.zip
  inflating: test/test/00000.jpg     
  inflating: test/test/00001.jpg     
  inflating: test/test/00002.jpg     
  inflating: test/test/00003.jpg     
  inflating: test/test/00004.jpg     
  inflating: test/test/00005.jpg     
  inflating: test/test/00006.jpg     
  inflating: test/test/00007.jpg     
  inflating: test/test/00008.jpg     
  inflating: test/test/00009.jpg     
  inflating: test/test/00010.jpg     
  inflating: test/test/00011.jpg     
  inflating: test/test/00012.jpg     
  inflating: test/test/00013.jpg     
  inflating: test/test/00014.jpg     
  inflating: test/test/00015.jpg     
  inflating: test/test/00016.jpg     
  inflating: test/test/00017.jpg     
  inflating: test/test/00018.jpg     
  inflating: test/test/00019.jpg     
  inflating: test/test/00020.jpg     
  inflating: test/test/00021.jpg     
  inflating: test/test/00022.jpg     
  inflating: test/test/00023.jpg     
  inflating: test/test/00024.jpg     
  inf

In [3]:
# LOAD TRAINING DATA
import numpy as np
from PIL import Image
from sklearn.datasets import load_files
path="train/train/"
data=load_files(path)
X=np.zeros((data.filenames.shape[0],28,28))
for i,file in enumerate(data.filenames):
    X[i,:]=np.array(Image.open(file))
y=data.target
# LOADING TEST DATA
import os
path="test/test/"
X_test=np.zeros((len(os.listdir(path)),28,28))
ID=[]
for i,file in enumerate(os.listdir(path)):
    X_test[i,:]=np.array(Image.open(path+file))
    ID.append(int(file.split(".")[0]))

## PROCESSING AND SPLITTING DATA

In [5]:
def skeletonize_data(X):
    X_skl_=np.zeros(X.shape)
    for i in range(X.shape[0]):
        img_=X[i,:]
        thres_=threshold_otsu(img_)
        img_binary_=img_>thres_
        img_skl_=skeletonize(img_binary_)
        X_skl_[i,:]=img_skl_
    return X_skl_

In [6]:
from skimage.filters import threshold_otsu
from skimage.morphology import skeletonize
from sklearn.model_selection import train_test_split
# Processing data
#X_skl=skeletonize_data(X)
# Splitting data into training and validation
X_train,X_val,y_train,y_val=train_test_split(X, y,test_size=0.2)

## CNN MODEL

In [8]:
28*28/8

98.0

In [16]:
from keras.models import Model, Sequential
from keras.constraints import maxnorm
from keras.layers import *

def make_model():
    model = Sequential()

    model.add(Conv2D(56, (3, 3),input_shape=(28, 28,1), activation='relu', padding='same'))
    model.add(GaussianNoise(0.2))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(56, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(SpatialDropout2D(0.1))

    model.add(Conv2D(112, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(112, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(SpatialDropout2D(0.1))


    model.add(Conv2D(224, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(224, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(SpatialDropout2D(0.1))
    
    model.add(Flatten())
    model.add(Dense(784, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(98, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    return model

In [17]:
# Creating the CNN model and printing summary
model=make_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 28, 28, 56)        560       
_________________________________________________________________
gaussian_noise_2 (GaussianNo (None, 28, 28, 56)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 28, 28, 56)        224       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 28, 56)        28280     
_________________________________________________________________
batch_normalization_10 (Batc (None, 28, 28, 56)        224       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 56)        0         
_________________________________________________________________
spatial_dropout2d_4 (Spatial (None, 14, 14, 56)       

In [18]:
# Creating optimizer and compiling model
from keras.optimizers import Adam
epochs=100
lrate=0.001
decay=lrate/100
optimizer=Adam(lr=lrate,decay=decay, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['accuracy']) 

In [19]:
# Creating an image augmentation object and fitting it
from tensorflow.keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.3)
aug.fit(X.reshape(X.shape[0],28,28,1))

In [20]:
import tensorflow as tf
checkpoint_filepath = 'checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [21]:
from keras.utils import np_utils
history=model.fit_generator(aug.flow(X_train.reshape(-1,28,28,1),
                                     y_train, batch_size=128), # np_utils.to_categorical(y_train)
                            steps_per_epoch = X_train.shape[0] / 128
                  ,epochs=epochs, validation_data=(X_val.reshape(-1,28,28,1),
                                                  y_val ), verbose=1, # np_utils.to_categorical(y_val)
                           callbacks=[model_checkpoint_callback]) 

Epoch 1/100
375/375 [==============================] - 238s 634ms/step - loss: 1.3676 - accuracy: 0.5147 - val_loss: 0.8746 - val_accuracy: 0.6750
Epoch 2/100
375/375 [==============================] - 232s 619ms/step - loss: 0.3603 - accuracy: 0.8761 - val_loss: 0.6392 - val_accuracy: 0.8104
Epoch 3/100
375/375 [==============================] - 234s 623ms/step - loss: 0.1980 - accuracy: 0.9346 - val_loss: 0.0807 - val_accuracy: 0.9730
Epoch 4/100
375/375 [==============================] - 237s 632ms/step - loss: 0.1498 - accuracy: 0.9496 - val_loss: 0.2054 - val_accuracy: 0.9339
Epoch 5/100
375/375 [==============================] - 232s 618ms/step - loss: 0.1219 - accuracy: 0.9611 - val_loss: 0.1699 - val_accuracy: 0.9445
Epoch 6/100
375/375 [==============================] - 235s 627ms/step - loss: 0.1095 - accuracy: 0.9636 - val_loss: 0.1943 - val_accuracy: 0.9438
Epoch 7/100
375/375 [==============================] - 234s 624ms/step - loss: 0.0942 - accuracy: 0.9701 - val_loss: 0

In [35]:
import time
model.load_weights(checkpoint_filepath)
model.save("model-100-csc-{}".format(time.time()))

In [36]:
y_test_pred=model.predict(X_test.reshape(-1,28,28,1))
y_test_pred

array([[8.82210443e-05, 5.87519980e-06, 1.71557003e-05, ...,
        2.40211248e-05, 1.03943103e-05, 9.99504924e-01],
       [9.99784768e-01, 4.52195127e-06, 2.89846957e-05, ...,
        1.44074329e-06, 5.87490058e-06, 3.16840965e-06],
       [9.99561131e-01, 4.04133652e-06, 6.01250467e-05, ...,
        3.19134620e-06, 1.56253736e-05, 2.70504529e-06],
       ...,
       [1.70095987e-03, 9.91613507e-01, 4.37200419e-04, ...,
        3.81620228e-03, 3.88355838e-05, 1.21916564e-04],
       [2.66796997e-05, 9.99790251e-01, 1.99962415e-05, ...,
        9.10121380e-05, 1.35633934e-06, 5.58420970e-06],
       [1.21938501e-06, 1.45865727e-06, 1.13697834e-05, ...,
        9.12971154e-06, 4.82189216e-06, 5.84918382e-07]], dtype=float32)

In [37]:
y=np.argmax(y_test_pred, axis=1)
y

array([9, 0, 0, ..., 1, 1, 3])

In [38]:
ids=np.array(ID)

In [ ]:
"""
import pandas as pd
df=pd.DataFrame(np.concatenate((ids.reshape(-1,1),y_test_pred.reshape(-1,1)), axis=1),
                columns=["Id","Category"])
df
"""

In [ ]:
#df.to_csv("submission.csv", index=False)

In [29]:
data.target_names, data.target

(['1', '10', '2', '3', '4', '5', '6', '7', '8', '9'],
 array([0, 3, 9, ..., 7, 7, 0]))

In [39]:
target_dict={}
for i,t in enumerate(data.target_names):
    target_dict[i]=t

In [41]:
y_real=[]
for Y in y:
    y_real.append(target_dict[Y])

In [42]:
y_real

['9',
 '1',
 '1',
 '4',
 '7',
 '3',
 '2',
 '3',
 '5',
 '7',
 '10',
 '3',
 '7',
 '7',
 '7',
 '9',
 '4',
 '10',
 '4',
 '6',
 '9',
 '2',
 '4',
 '7',
 '9',
 '10',
 '3',
 '1',
 '9',
 '4',
 '7',
 '6',
 '6',
 '7',
 '8',
 '7',
 '1',
 '6',
 '6',
 '4',
 '1',
 '2',
 '7',
 '8',
 '9',
 '9',
 '5',
 '5',
 '2',
 '10',
 '7',
 '8',
 '2',
 '6',
 '6',
 '6',
 '6',
 '3',
 '1',
 '8',
 '5',
 '9',
 '3',
 '3',
 '2',
 '2',
 '5',
 '6',
 '9',
 '7',
 '1',
 '7',
 '1',
 '10',
 '10',
 '1',
 '1',
 '5',
 '6',
 '5',
 '10',
 '4',
 '2',
 '2',
 '3',
 '9',
 '2',
 '3',
 '3',
 '3',
 '6',
 '4',
 '3',
 '4',
 '2',
 '1',
 '7',
 '8',
 '10',
 '7',
 '8',
 '5',
 '3',
 '10',
 '10',
 '3',
 '9',
 '5',
 '10',
 '1',
 '3',
 '1',
 '2',
 '10',
 '4',
 '8',
 '2',
 '9',
 '3',
 '10',
 '6',
 '7',
 '6',
 '5',
 '7',
 '10',
 '1',
 '9',
 '8',
 '10',
 '7',
 '1',
 '8',
 '8',
 '9',
 '1',
 '7',
 '2',
 '6',
 '10',
 '5',
 '5',
 '8',
 '1',
 '4',
 '9',
 '10',
 '10',
 '10',
 '3',
 '6',
 '5',
 '9',
 '9',
 '1',
 '2',
 '5',
 '8',
 '9',
 '1',
 '5',
 '8',
 '5',
 '5

In [44]:
import pandas as pd
res=pd.DataFrame(data=ids, columns=["Id"])
res["Category"]=y_real
res

,Id,Category
0,0,9
1,1,1
2,2,1
3,3,4
4,4,7
...,...,...
9995,9995,1
9996,9996,7
9997,9997,10
9998,9998,10


In [45]:
res.to_csv("submission.csv", index=False)

In [47]:
model.evaluate(X_val.reshape(-1,28,28,1),y_val)

12000/12000 [==============================] - 16s 1ms/step


[0.013001756483417315, 0.9960833191871643]